In [ ]:
import pandas as pd
import numpy as np

In [ ]:
# Read in dataset from (1)
uni = pd.read_csv('EM_full.csv')

In [ ]:
# Change row names
uni.rename(columns = {'conm':'name','sedol':'id','buy_date':'date', 'curcdd':'currency','prccd':'price','cshoc':'shares','atq':'assets','ltq':'liabilities'
                    ,'niq':'net income','oiadpq':'ebit','revtq':'revenue'}, inplace = True)

In [ ]:
# Set date to datetime type
uni['date'] = pd.to_datetime(uni['date'])

In [ ]:
# Drop unwanted columns from dataset
uni = uni[['name',
 'id',
 'date',
 'currency',
 'price',
 'shares',
 'assets',
 'liabilities',
 'net income',
 'ebit',
 'revenue']]

In [ ]:
# Drop rows where certain columns are zero
uni = uni[uni['revenue']!=0]
uni = uni[uni['assets']!=0]
uni = uni[uni['liabilities']!=0]
uni = uni[uni['price']!=0]
uni = uni[uni['shares']!=0]

In [ ]:
# Calculate sharehlders equity
uni['equity'] = uni['assets'] - uni['liabilities']

In [ ]:
# Sort data by date
uni = uni.sort_values(['date'], ascending=(True)).reset_index(drop=True)
# Calculate returns, investment and SGI
uni['return'] = uni.groupby('name')['price'].pct_change()
uni['investment'] = uni.groupby('name')['assets'].pct_change()
uni['sgi'] = uni.groupby('name')['revenue'].pct_change() + 1

In [ ]:
# Calculate market cap
uni['market cap'] = uni['price']*uni['shares']

# Now capping returns

In [ ]:
# Get unique list of dates
dates1 = uni['date'].unique().tolist()

In [ ]:
# This section finds the 5th and 95th percentile of the returns at each quarter
max_key = {}
min_key = {}

for d in dates1:
    temp = uni[uni['date']==d]
    if len(temp['return'].dropna())>0:
        mx = np.percentile(temp['return'].dropna(),95)
        mn = np.percentile(temp['return'].dropna(),5)
    else:
        mx = mn = np.nan
    max_key[d] = mx
    min_key[d] = mn

In [ ]:
# This section constrains returns between 5% and 95%
for i in range(len(uni)):
    
    date = uni.iloc[i,2]

    mx = max_key[date]
    mn = min_key[date]
    if uni.iloc[i,-4] > mx:
        uni.iloc[i,-4] = mx
    if uni.iloc[i,-4] < mn:
        uni.iloc[i,-4] = mn

# Back to vriables

In [ ]:
# Drop rows with the same name on the same date keeping the first 
uni = uni.drop_duplicates(['name','date'],keep= 'first')

In [ ]:
# Calculate book to market value
uni['b/m'] = uni['equity']/uni['market cap']

In [ ]:
# Drop rows where certain columns are zero
uni = uni[uni['equity']!=0]
uni = uni[uni['assets']!=0]
uni = uni[uni['market cap']!=0]

In [ ]:
# Calculate roa, roe and debt-to-equity 
uni['roa'] = uni['net income']/uni['assets']
uni['roe'] = uni['net income']/uni['equity']
uni['d/e'] = uni['liabilities']/uni['equity']

In [ ]:
# Get mean return of market at each period. I.E. the market return
market_ret = uni.groupby('date')['return'].mean()

In [ ]:
# Set index as date
uni.set_index('date', inplace=True, drop=False)

In [ ]:
# Join main data with the market mean return at each period
uni = uni.join(market_ret, how='left' , lsuffix='', rsuffix='_market')

In [ ]:
# Calculate alpha
uni['alpha'] = uni['return'] - uni['return_market']

In [ ]:
# Set index as id and sort the values by date then name
uni.set_index('id', inplace=True, drop=False)
uni = uni.sort_values(['date','name'], ascending=(True,True)).reset_index(drop=True)

In [ ]:
# Get rolling mean return and volatility of returns
grouped = uni.groupby('name')
uni['mean_return'] = grouped['return'].rolling(window=12, min_periods=4).mean().reset_index(0, drop=True)
uni['vol'] = grouped['return'].rolling(window=12, min_periods=4).std().reset_index(0, drop=True)

In [ ]:
# sort values by name then date
uni = uni.sort_values(['name','date'], ascending=(True,True))

In [ ]:
# Drop unwanted columns then reset index
uni = uni[['name', 'id', 'date', 'alpha', 'market cap', 'b/m', 'ebit','investment', 'vol','mean_return','roa','roe','sgi','d/e','return']].reset_index(drop=True)

# Fama French Factor Loadings

In [ ]:
# Read emerging markets factor data
ff = pd.read_csv('EM_factors.csv', index_col='Date')

In [ ]:
# Get the risk free rate in the right format
ff['RF']=ff['RF']/100

In [ ]:
# Work out quarterly risk-free-rate from monthly risk-free-rate
ff['RF']=ff['RF']+1

for i in range(5, len(ff), 3):
    ff.iloc[i,-1] = ff.iloc[i-2:i+1,-1].prod()-1

In [ ]:
# Get date to join factors data to main data
uni['ff_date'] = 0
for i in range(len(uni)):
    uni.iloc[i,-1] = uni.iloc[i,2].strftime("%Y%m")

In [ ]:
# Set index as string for factor data
ff.index = ff.index.astype(str)

In [ ]:
# Set index for main data as date
uni.index = uni['ff_date']

In [ ]:
# Join main data to factor data
uni = uni.join(ff,how="left",lsuffix='', rsuffix='')

In [ ]:
# Drop unwanted data from main data
uni = uni[['name','id','date','alpha','market cap','b/m','ebit','investment','vol','mean_return','roa',
 'roe','sgi','d/e','return','Mkt-RF','SMB','HML','RMW','CMA','RF']]

In [ ]:
# Calculate excess returns
uni['excess_ret'] = uni['return']-uni['RF']

In [ ]:
# Make dataframe to store factor loadings
betas = uni.copy()
betas['date']=0
betas = betas[['date', 'name', 'Mkt-RF', 'SMB', 'HML', 'RMW','CMA']]
betas['intercept'] = np.nan
betas=betas[betas['name']== 90]

In [ ]:
# Get list of sorted dates
dates = sorted(uni['date'].unique().tolist())

In [ ]:
# Calculate factor loadings for data
# A 36 month period was used, all 36 periods werent available a minimum of 12 would be taken 
import statsmodels.api as sm

for i in range(4, len(dates)+1):
    print(dates[i-1])
    if i < 12: 
        temp = uni[uni['date'].isin(dates[:i])]
        
    elif i >= 12:

        temp = uni[uni['date'].isin(dates[i-12:i])]

    temp['intercept'] = 1
    names = temp['name'].unique().tolist()
    for n in names:
        train = temp[temp['name'] == n][['excess_ret','Mkt-RF', 'SMB', 'HML', 'RMW','CMA', 'intercept']].dropna()
        if len(train) > 4:
            
            model = sm.OLS(train['excess_ret'], train[['Mkt-RF', 'SMB', 'HML', 'RMW','CMA', 'intercept']].dropna()).fit()
            betas.loc[len(betas)] = [dates[i-1], n,  model.params[0], model.params[1], model.params[2], model.params[3], model.params[4], model.params[5]]


In [ ]:
# Make a key to join main data and factor loadings
betas.index = betas['date'].astype(str)+betas['name']
uni.index = uni['date'].astype(str)+uni['name']

In [ ]:
# Join the data
uni = uni.join(betas, how='left' , lsuffix='', rsuffix='_loading')

# Shift Returns

In [ ]:
# Get list of dates
dates = uni['date'].unique().tolist()

In [ ]:
# Get dictionary to lag data
dic = {}

for i in range(len(dates)-1):
        dic[dates[i+1]] = dates[i]

In [ ]:
# Set last date as date
uni['last_date'] = uni['date']

In [ ]:
# Move returns one period backwards

for i in range(len(uni)):
    if uni.iloc[i,-1] in dic:
        uni.iloc[i,-1] = dic[uni.iloc[i,-1]]
    else:
        uni.iloc[i,-1] = np.nan

In [ ]:
# Make adjusted returns a copy of main data
returns_adjust = uni.copy()

In [ ]:
# Drop unwanted columns
returns_adjust = returns_adjust[['name', 'return', 'last_date', 'RF', 'excess_ret']]

In [ ]:
# Get key to join adjusted returns and main data
uni.index = uni['date'].astype(str) + uni['name']
returns_adjust.index = returns_adjust['last_date'].astype(str) + returns_adjust['name']

In [ ]:
# Join the main data to adjusted returns data
uni = uni.join(returns_adjust[['return', 'RF', 'excess_ret']], how='left' , lsuffix='', rsuffix='_adjusted')

In [ ]:
# Sort values by name then date
uni = uni.sort_values(['name','date'], ascending=(True,True)).reset_index(drop=True)

In [ ]:
# Drop unwanted columns
uni = uni[['name','id','date', 'alpha', 'market cap', 'b/m', 'ebit','investment', 'vol', 'mean_return', 'roa', 'roe',
 'sgi', 'd/e', 'Mkt-RF', 'SMB', 'HML', 'RMW', 'CMA', 'Mkt-RF_loading', 'SMB_loading', 'HML_loading', 'RMW_loading', 'CMA_loading','intercept',
           'return_adjusted','RF_adjusted','excess_ret_adjusted']]

# Back to variables

In [ ]:
# Set uni2 as a copy of uni1
uni2 = uni.copy()

In [ ]:
# Drop rows where columns are nan 
uni2 = uni2[uni2['alpha'].notna()]
uni2 = uni2[uni2['market cap'].notna()]
uni2 = uni2[uni2['b/m'].notna()]
uni2 = uni2[uni2['ebit'].notna()]
uni2 = uni2[uni2['investment'].notna()]
uni2 = uni2[uni2['vol'].notna()]
uni2 = uni2[uni2['mean_return'].notna()]
uni2 = uni2[uni2['roa'].notna()]
uni2 = uni2[uni2['roe'].notna()]
uni2 = uni2[uni2['sgi'].notna()]
uni2 = uni2[uni2['d/e'].notna()]
uni2 = uni2[uni2['return_adjusted'].notna()]

uni2 = uni2[uni2['Mkt-RF_loading'].notna()]
uni2 = uni2[uni2['SMB_loading'].notna()]
uni2 = uni2[uni2['HML_loading'].notna()]
uni2 = uni2[uni2['RMW_loading'].notna()]
uni2 = uni2[uni2['CMA_loading'].notna()]
uni2 = uni2[uni2['intercept'].notna()]

# Finishing touches

In [ ]:
# Set index as date
uni2.index = uni2['date']

In [ ]:
# Drop rows with nan values
uni2 = uni2.dropna()

In [ ]:
# Drop rows where name is duplicated on the same date and keep the first
uni2 = uni2.drop_duplicates(['name','date'],keep= 'first')

In [ ]:
# Rename the columns 
uni2.rename(columns = {'alpha':'Alpha','market cap':'Market Capitalisation','b/m':'Book-to-Market Ratio','ebit':'EBIT'
                     ,'investment':'Investment','vol':'Stock Price Volatility','mean_return':'Mean Return'
                     ,'roa':'ROA','roe':'ROE','sgi':'SGI','d/e':'Debt-to-Equity Ratio','Mkt-RF_loading':'Market Risk Factor Loading','SMB_loading':'SMB Factor Loading'
                     ,'HML_loading':'HML Factor Loading','RMW_loading':'RMW Factor Loading','CMA_loading':'CMA Factor Loading'}, inplace = True)

In [ ]:
# Reset index of main data
uni2 = uni2.reset_index(drop=True)

In [ ]:
# Sort values by name and then date
uni2 = uni2.sort_values(['name','date'], ascending=(True,True)).reset_index(drop=True)

In [ ]:
# Drop data to csv file
uni2.to_csv("EM_universe.csv", index = False)